### This willl be an active workbook looking at particular node and node measures, for example seeing affects pn strongest nodes - or how centrality changes etc, at the moment it is not developed and need to add for many different metrics and consider best ways to compare 

#### the first cell does the usual importing of packages - setting variables and providing subject dictionaries, I then load the graph and comp dicts, the rest is unstructured at the moment 

In [1]:
# this cell imports the necessary packages - sets variables to point to string keys - sets the sub case and control
# dictionaries as well as creating lists of global node and othe metrics


import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle

# set variables that will point to dictionary keys - with the type/output will be in the dict, note these 
# are not based on the output of the analagous networkx function but how I have written into dict


#data/matrices
graph = "graph" #networkx graph basically dictionaries all the way down
data = "data" #pandas data frame 
bin_mat = "binary_matrix" #numpy array 
bin_df = "binary_df" # pandas data frame 
data_numpy = "data_as_numpy_array" #numpy array
norm_lap = "normalised_laplacian" #numpy array?

#global metrics 
gl_eff = "global_efficiency" #float
loc_eff = "local_efficiency" #float
clus_coef = "clusetering_coefficent" #float
ave_path_length = "ave_path_length" #float
clique_num = "clique_number" #int (poss float but always integer valued)
ass_coef = "assortativity_coefficient" #float
transitivity = "transitivity" #float
rich_club_coeff = "rich_club_coefficient" #currently not working would be float
sw_sigma = "" #currently not working would be float
sw_gamma = "" #currently not working would be float
ave_strength = "average_node_strength" #float
no_edges = "number_of_edges" #int
ave_deg = "average_degree" #int

#individual node/edge metrics and dicts
deg = "degree" #dictionary - nodes as key degree as values 
bet_cent = "betweenness_centrality" # dictionary - nodes as key betweeness as values 
edge_bet_cent = "edge_betweenness_centrality" # dictionary - edge tuple as key edgebetweeness centrality as values 
eigen_cent = "eigenvector_centrality" # dictionary - nodes as eigenvalue centrality as values 
deg_cent = "degree_centrality" # dictionary - nodes as key degree centrality as values 
strength = "node_strength"  # dictionary - nodes as key strength as values 

#other
deg_hist = "degree_histogram" #list - index is degree value is how many nodes with that degree 
norm_lap_spec = "normalised_laplacian_spectrum" #list of floats?


#subject dictionary with a list of runs - corresponds to how connectome file is saved for now hand written 
sub_dict = {"NENAH004" : ["orig", "run2"], "NENAH010" :  ["orig", "run2"], "NENAH011" :  ["orig", "run2"],
                "NENAH012" : ["orig", "run2"], "NENAH015" : ["orig", "run2"], "NENAH016" : ["orig", "run2"], 
                "NENAH022" : ["orig", "run2"], "NENAH024" : ["orig", "run2"], "NENAHC002" : ["orig", "run2", "run3"], 
                "NENAHC003" : ["orig", "run2","run3"], "NENAHC010" : ["orig", "run2", "run3"],
                "NENAHC013" : ["orig", "run2", "run3"], "NENAHC016" : ["orig", "run2", "run3"],
                "NENAHC018" : ["orig", "run2", "run3"], "NENAHC020": ["orig", "run2", "run3"],
                "NENAHC025" : ["orig", "run2"]
           }

case_dict = {"NENAH004" : ["orig", "run2"], "NENAH010" :  ["orig", "run2"], "NENAH011" :  ["orig", "run2"],
                "NENAH012" : ["orig", "run2"], "NENAH015" : ["orig", "run2"], "NENAH016" : ["orig", "run2"], 
                "NENAH022" : ["orig", "run2"], "NENAH024" : ["orig", "run2"]
            }

control_dict = { "NENAHC002" : ["orig", "run2", "run3"],  "NENAHC003" : ["orig", "run2", "run3"], 
                "NENAHC010" : ["orig", "run2", "run3"],"NENAHC013" : ["orig", "run2", "run3"],
                "NENAHC016" : ["orig", "run2", "run3"], "NENAHC018" : ["orig", "run2", "run3"],
                "NENAHC020": ["orig", "run2", "run3"], "NENAHC025" : ["orig", "run2",]
             }

#create the lists which might be usuefull in automating data frame creation
#note these should be strings
global_metrics = [gl_eff, loc_eff, clus_coef, ave_path_length, clique_num,
                  transitivity, ave_strength, no_edges, ave_deg] 

node_metrics = [deg, strength, bet_cent, eigen_cent, deg_cent]

other_metrics = [deg_hist, norm_lap_spec, edge_bet_cent,]


In [2]:
# load in the main graph dictionary and they created comparison dictionary with various thresholds

with open('graph_dict.pickle', 'rb') as handle:
     graph_dict = pickle.load(handle)
        
with open('comp_dict.pickle', 'rb') as handle:
    comp_dict = pickle.load(handle)

In [13]:
str_df = pd.read_csv('node_strength.csv', index_col = 0)
str_df.drop(index=["NENAHC002run3",  "NENAHC003run3", "NENAHC010run3", "NENAHC013run3", "NENAHC016run3", "NENAHC018run3",
           "NENAHC020run3"], axis =0, inplace=True)
str_df.head(40)

,node 0,node 1,node 2,node 3,node 4,node 5,node 6,node 7,node 8,node 9,...,node 74,node 75,node 76,node 77,node 78,node 79,node 80,node 81,node 82,node 83
sub-id,,,,,,,,,,,,,,,,,,,,,
NENAH004orig,0.549006,0.968215,0.951273,0.800040,0.735932,0.514218,0.851629,0.581491,1.129222,0.751237,...,0.811117,1.254912,0.937596,0.641393,0.827985,0.317535,0.728092,0.216967,1.667508,0.278407
NENAH004run2,0.544679,0.968612,0.947988,0.804754,0.749404,0.518200,0.848783,0.571631,1.128087,0.750347,...,0.805063,1.252816,0.927042,0.638589,0.838002,0.332214,0.743250,0.220890,1.651871,0.277616
NENAH010orig,0.676872,0.940612,1.306808,1.439880,0.783067,0.612366,1.172659,0.860758,1.460994,1.026468,...,0.846541,1.993300,1.351539,0.743405,1.246134,0.513559,0.493913,0.335691,2.415398,0.348669
NENAH010run2,0.669661,0.964943,1.308211,1.423303,0.766357,0.614096,1.171095,0.862126,1.475563,1.015932,...,0.855740,1.989071,1.328282,0.749338,1.241012,0.519827,0.505971,0.344614,2.406425,0.348373
NENAH011orig,0.580981,0.809830,0.989932,1.276166,0.652961,0.538555,0.908333,0.713531,1.357086,0.717344,...,0.907890,1.377530,1.268173,0.636338,0.878582,0.581541,0.598559,0.272136,1.995659,0.225831
NENAH011run2,0.570289,0.831574,1.006684,1.297420,0.658020,0.528901,0.911839,0.705752,1.355096,0.716615,...,0.912385,1.362108,1.271217,0.638845,0.872452,0.582729,0.579033,0.278776,1.986059,0.227849
NENAH012orig,0.644419,1.135882,1.211711,1.236519,1.050785,0.622076,1.136809,0.704054,1.562131,0.833175,...,1.082606,1.423801,1.550698,0.680276,1.077925,0.482015,0.697853,0.210962,2.684315,0.261223
NENAH012run2,0.647616,1.131909,1.200566,1.253456,1.049786,0.626538,1.128960,0.698691,1.543134,0.828163,...,1.069409,1.419646,1.540976,0.684937,1.080548,0.476949,0.689523,0.213312,2.704998,0.262188
NENAH015orig,0.514286,0.901285,0.993228,1.700861,0.659720,0.767941,1.138115,0.664640,1.839055,0.907127,...,0.768325,1.646160,1.358964,0.697692,0.795924,0.324113,0.624691,0.214250,2.049510,0.305641


In [50]:
subjects= ["NENAH004" , "NENAH010" , "NENAH011", "NENAH012", "NENAH015", "NENAH016",
           "NENAH022", "NENAH024", 
           "NENAHC002",  "NENAHC003", "NENAHC010", "NENAHC013", "NENAHC016", "NENAHC018",
           "NENAHC020", "NENAHC025", ]


str_dict = {}
for i, sub in enumerate(subjects) :
    str_dict[sub] ={}
    df = str_df.iloc[[2*i, 2*i+1]]
    df_orig = df.drop(f'{sub}run2').T
    df_run2 = df.drop(f'{sub}orig').T
    df_orig_sorted = df_orig.sort_values(f'{sub}orig', ascending = False)
    df_run2_sorted = df_run2.sort_values(f'{sub}run2', ascending = False)
    str_dict[sub]['orig'] = df_orig_sorted
    str_dict[sub]['run2'] = df_run2_sorted


#     print(df_orig_sorted)

str_dict['NENAH010']['orig'].head(15)
# str_dict['NENAH010']['run2'].head(15)

sub-id,NENAH010orig
node 42,3.156311
node 43,2.892258
node 44,2.773933
node 35,2.728196
node 36,2.509406
node 82,2.415398
node 37,2.411396
node 71,2.242611
node 33,2.121894
node 22,2.080637


In [33]:
str_dict['NENAHC025']['orig'].head(15)
# str_dict['NENAHC025']['run2'].head(15)

sub-id,NENAHC025orig
node 35,3.350337
node 42,3.197709
node 37,3.013794
node 44,3.005262
node 82,2.350277
node 33,2.267655
node 36,2.231314
node 71,2.228641
node 22,2.200611
node 43,2.175603


In [39]:
str_dict['NENAH011']['orig'].head(15)
str_dict['NENAH011']['run2'].head(15)

sub-id,NENAH011run2
node 36,2.834564
node 43,2.633831
node 35,2.455549
node 22,2.272719
node 42,2.234751
node 82,1.986059
node 38,1.960674
node 46,1.915119
node 71,1.823334
node 33,1.819261


In [55]:
subjects= ["NENAH004" , "NENAH010" , "NENAH011", "NENAH012", "NENAH015", "NENAH016",
           "NENAH022", "NENAH024", 
           "NENAHC002",  "NENAHC003", "NENAHC010", "NENAHC013", "NENAHC016", "NENAHC018",
           "NENAHC020", "NENAHC025", ]

bet_df = pd.read_csv('betweenness_centrality.csv', index_col = 0 )
bet_df.drop(index=["NENAHC002run3",  "NENAHC003run3", "NENAHC010run3", "NENAHC013run3", "NENAHC016run3", "NENAHC018run3",
           "NENAHC020run3"], axis =0, inplace=True)
# bet_df.head(40)

bet_dict = {}
for i, sub in enumerate(subjects) :
    bet_dict[sub] ={}
    df = bet_df.iloc[[2*i, 2*i+1]]
    df_orig = df.drop(f'{sub}run2').T
    df_run2 = df.drop(f'{sub}orig').T
    df_orig_sorted = df_orig.sort_values(f'{sub}orig', ascending = False)
    df_run2_sorted = df_run2.sort_values(f'{sub}run2', ascending = False)
    bet_dict[sub]['orig'] = df_orig_sorted
    bet_dict[sub]['run2'] = df_run2_sorted


#     print(df_orig_sorted)

bet_dict['NENAH004']['orig'].head(15)
# bet_dict['NENAH004']['run2'].head(15)

sub-id,NENAH004orig
node 35,0.019505
node 37,0.016098
node 44,0.014220
node 12,0.013834
node 23,0.013613
node 72,0.013276
node 42,0.013253
node 33,0.013088
node 27,0.012766
node 26,0.012424


In [41]:
bet_df.drop(index=["NENAHC002run3",  "NENAHC003run3", "NENAHC010run3", "NENAHC013run3", "NENAHC016run3", "NENAHC018run3",
           "NENAHC020run3"], axis =0, inplace=True)
bet_df.head(40)

,node 0,node 1,node 2,node 3,node 4,node 5,node 6,node 7,node 8,node 9,...,node 74,node 75,node 76,node 77,node 78,node 79,node 80,node 81,node 82,node 83
sub-id,,,,,,,,,,,,,,,,,,,,,
NENAH004orig,0.000233,0.002419,0.004343,0.001721,0.000424,0.006057,0.007693,0.004680,0.008992,0.008910,...,0.008444,0.009602,0.009369,0.003254,0.000680,0.001213,0.005684,0.000413,0.011190,0.004080
NENAH004run2,0.000129,0.002111,0.004479,0.001309,0.000544,0.003281,0.007667,0.003653,0.010453,0.008127,...,0.008430,0.012649,0.009832,0.002083,0.000671,0.000885,0.004204,0.000284,0.010344,0.005046
NENAH010orig,0.000104,0.002262,0.002968,0.000455,0.000491,0.001811,0.003926,0.005075,0.005772,0.003639,...,0.005154,0.005499,0.008375,0.003866,0.002820,0.001139,0.001081,0.000719,0.007580,0.005263
NENAH010run2,0.000088,0.002695,0.002246,0.000805,0.000665,0.002794,0.003491,0.005566,0.004759,0.003922,...,0.004537,0.007979,0.007612,0.003787,0.002956,0.001239,0.000983,0.000286,0.006368,0.004945
NENAH011orig,0.000039,0.001133,0.005811,0.001495,0.000234,0.002387,0.006177,0.003581,0.009571,0.008791,...,0.008161,0.013963,0.014235,0.004182,0.002519,0.002538,0.001929,0.000328,0.013852,0.009867
NENAH011run2,0.000139,0.001321,0.004646,0.001207,0.000676,0.001685,0.007064,0.002612,0.008281,0.006910,...,0.008936,0.014816,0.014112,0.004870,0.001833,0.002722,0.001715,0.000020,0.016115,0.007294
NENAH012orig,0.000179,0.001094,0.003016,0.000942,0.000260,0.003039,0.007719,0.004874,0.010833,0.003642,...,0.007288,0.007816,0.009270,0.005015,0.001932,0.004290,0.001281,0.000416,0.014004,0.003434
NENAH012run2,0.000266,0.001007,0.003722,0.000453,0.000666,0.003238,0.009404,0.003949,0.009023,0.003087,...,0.008487,0.005163,0.008279,0.005118,0.002983,0.005239,0.002529,0.000357,0.009892,0.003033
NENAH015orig,0.000065,0.002455,0.001852,0.000565,0.000551,0.003993,0.004070,0.005893,0.004994,0.006395,...,0.004682,0.006612,0.008589,0.005085,0.003325,0.000714,0.003275,0.000189,0.008618,0.004307
